<a href="https://colab.research.google.com/github/RyutaroHashimoto/gigwork_quest_blue/blob/katsuta/Image_Recognition_AI_GIG_Work_Quest_1_Team_Brue_EffNetB0_with_Rand_Augumen_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
# # !pip install tensorflow-addons==0.6.0
# # !pip install tensorflow-io==0.9.1
# # !pip install efficientnet
!pip install -U git+https://github.com/qubvel/efficientnet

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-oxdz52nx
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-oxdz52nx
  Created wheel for efficientnet: filename=efficientnet-1.1.1-cp37-none-any.whl size=18421 sha256=03fb2b424d0ab71864a9bf443c455b396034b25d51553d95dc44316eff5b62a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-hs27l1yk/wheels/64/60/2e/30ebaa76ed1626e86bfb0cc0579b737fdb7d9ff8cb9522663a
Successfully built efficientnet
  Found existing installation: efficientnet 1.1.1
    Uninstalling efficientnet-1.1.1:
      Successfully uninstalled efficientnet-1.1.1


In [2]:
!nvidia-smi

Fri Mar 19 11:49:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

 # Import library

In [3]:
# ライブラリの読み込み

import os
import sys
import random
import shutil
import gc
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import keras
import keras.models as M
import keras.layers as L
import keras.backend as K
import tensorflow as tf
from keras.datasets import cifar100
from skimage.io import imread
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras import optimizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from keras.constraints import max_norm
from keras.applications.imagenet_utils import decode_predictions
from PIL import Image, ImageEnhance, ImageOps
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from efficientnet.keras import EfficientNetB0
from efficientnet.keras import EfficientNetB1
from efficientnet.keras import EfficientNetB2
from efficientnet.keras import EfficientNetB7
from efficientnet.keras import center_crop_and_resize, preprocess_input
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, GlobalMaxPooling2D, GlobalAveragePooling2D,Input
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Input,GlobalAveragePooling2D,BatchNormalization

In [4]:
# ディレクトリ移動
%cd drive/MyDrive/AI Quest

/content/drive/MyDrive/AI Quest


In [5]:
!ls

 dataset				   Submit
'Image Recognition_Efn.png'		   submit_EfficientNet.csv
 label_master.tsv			   train_master.tsv
 sample_submit_Adam_32_100_1.2700348.csv   X_test.npy
 sample_submit.csv			   X_train.npy
 sample_submit_RMSprop_32_100.csv	   Y_train.npy
 SaveModels


# Reading Data

In [6]:
labels = pd.read_csv("train_master.tsv", sep="\t") #教師ラベルの読み込み

In [7]:
labels.head()

,file_name,label_id
0,train_00000.png,11
1,train_00001.png,15
2,train_00002.png,4
3,train_00003.png,14
4,train_00004.png,1


In [8]:
master = pd.read_csv("label_master.tsv", sep="\t") #ラベルマスタの読み込み

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
master.head()

,label_id,label_name
0,0,aquatic_mammals
1,1,fish
2,2,flowers
3,3,food_containers
4,4,fruit_and_vegetables


In [11]:
sample = pd.read_csv("sample_submit.csv", header=None, sep=",")

In [12]:
sample.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,test_00000.png,0.036151,0.014607,0.034591,0.079934,0.053778,0.011690,0.039329,0.079885,0.047027,0.055246,0.062593,0.037947,0.080989,0.066377,0.020849,0.057315,0.056351,0.033405,0.078505,0.053430
1,test_00001.png,0.033393,0.037818,0.021027,0.043661,0.082885,0.041105,0.069309,0.059735,0.023879,0.001008,0.077569,0.053666,0.044468,0.040490,0.090077,0.050215,0.052718,0.067940,0.030642,0.078394
2,test_00002.png,0.098902,0.047091,0.022525,0.059422,0.038273,0.011877,0.086344,0.054882,0.046612,0.050381,0.028493,0.027183,0.092749,0.076549,0.026914,0.099406,0.032409,0.038710,0.000611,0.060666
3,test_00003.png,0.054705,0.080344,0.047048,0.083701,0.043293,0.068830,0.026523,0.076391,0.008649,0.071797,0.020802,0.054486,0.065079,0.060174,0.045692,0.052243,0.006735,0.036344,0.070731,0.026432
4,test_00004.png,0.087472,0.045188,0.091187,0.003265,0.110322,0.102048,0.048273,0.080005,0.050142,0.018821,0.096636,0.038774,0.050342,0.044085,0.057429,0.023512,0.018324,0.002596,0.030469,0.001106


In [13]:
# X_train = np.load('X_train.npy')

In [14]:
# Y_train = np.load('Y_train.npy')

In [15]:
# X_test = np.load('X_test.npy')

In [16]:
(X_train, Y_train), (X_test, Y_test) = cifar100.load_data(label_mode='fine')

In [17]:
Y_train

array([[19],
       [29],
       [ 0],
       ...,
       [ 3],
       [ 7],
       [73]])

# Normalization

In [18]:
# train = X_train / 255
# test = X_test / 255

train = X_train
test = X_test

# Splitting Data

In [19]:
val_index_list = []
st = StratifiedShuffleSplit(n_splits = 2, test_size = 0.2, random_state = 1)
for train_index, val_index in st.split(train, Y_train):
    val_index_list.append(val_index)
    X_train_image, X_val_image, y_train_label, y_val_label = train[train_index], train[val_index], Y_train[train_index], Y_train[val_index]
    df_val = labels.iloc[val_index]
print("The number of training data : ", X_train_image.shape[0])
print("The number of validation data : ", X_val_image.shape[0])

The number of training data :  40000
The number of validation data :  10000


In [20]:
df_val

,file_name,label_id
20172,train_20172.png,13
38255,train_38255.png,18
15501,train_15501.png,5
14108,train_14108.png,0
18794,train_18794.png,18
...,...,...
43061,train_43061.png,17
5797,train_05797.png,3
16251,train_16251.png,10
13396,train_13396.png,11


In [21]:
# # Spliting the training data into a training data and a validation data.
# st = StratifiedShuffleSplit(n_splits = 2, test_size = 0.2, random_state = 1)
# for train_index, val_index in st.split(train, Y_train):
#     X_train_image, X_val_image, y_train_label, y_val_label = train[train_index], train[val_index], Y_train[train_index], Y_train[val_index]
    
# print("The number of training data : ", X_train_image.shape[0])
# print("The number of validation data : ", X_val_image.shape[0])

# Setting Callbacks

In [22]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1
)

In [23]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    # patience=3,
    verbose=1
)

In [24]:
# ModelCheckpoint
weights_dir='/content/drive/MyDrive/AI Quest/SaveModels/'
if os.path.exists(weights_dir)==False:os.mkdir(weights_dir)
model_checkpoint = ModelCheckpoint(
    weights_dir + "val_loss{val_loss:.3f}.hdf5",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    period=3
)


In [25]:
def step_decay(epoch):
    lr = 0.001
    if(epoch >= 100):
        lr/=5
    if(epoch>=140):
        lr/=2
    return lr

In [26]:
lr_decay = LearningRateScheduler(step_decay)

# Setting Various Parameters

In [27]:
batch_size = 32
epochs = 100

steps_per_epoch  = X_train_image.shape[0] // batch_size
validation_steps = X_val_image.shape[0] // batch_size

# Data Augmentation

In [28]:
datagen = ImageDataGenerator(
    featurewise_center = False,
    samplewise_center = False,
    featurewise_std_normalization = False,
    samplewise_std_normalization = False,
    zca_whitening = False,
    rotation_range = 0,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    vertical_flip = False
)

# Fine Tuning by EfficientNetB0

### （１）Resizing images

In [29]:
inputs = tf.keras.Input(shape=(None, None, 3))
x = tf.keras.layers.Lambda(lambda img: tf.image.resize(img, (224, 224)))(inputs)
# x = tf.keras.layers.Lambda(EfficientNetB0.)(x)

### （２）Loading the trained model

In [30]:
# from efficientnet import EfficientNetB0
input_shape = (224, 224, 3)
base_model = EfficientNetB0(
                            input_shape=input_shape,
                            weights='imagenet',
                            include_top=False,
                            # drop_connect_rate=1.0, #正則化 defalt値は0.2
                            # drop_connect_rate=0.6, #正則化 defalt値は0.2
                            input_tensor=x)


### （３）Rebuild the model

In [31]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
# x = Dropout(rate=0.5)(x)
x = Dropout(rate=0.25)(x)
predictions = Dense(100, activation='softmax')(x)

In [32]:
# base_model.trainable = False

In [33]:
# ネットワーク定義
model = Model(inputs = base_model.input, outputs = predictions)

In [34]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         lambda[0][0]                     
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
______________________________________________________________________________________________

### （４）Training the model

In [35]:
y_train_categorical = keras.utils.to_categorical(y_train_label)
y_val_categorical = keras.utils.to_categorical(y_val_label)

In [36]:
y_train_categorical.shape

(40000, 100)

In [37]:
y_val_label.shape

(10000, 1)

In [38]:
# Compile
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# %%time

hist = model.fit_generator(
                           datagen.flow(X_train_image, y_train_label, batch_size = batch_size),
                           steps_per_epoch = X_train_image.shape[0] // batch_size,
                           epochs = epochs,
                           validation_data = (X_val_image, y_val_label),
                          #  callbacks = [reduce_lr],
                          #  callbacks = [early_stopping, reduce_lr, model_checkpoint],
                           callbacks = [early_stopping, reduce_lr],
                           shuffle = True,
                           verbose = 1)

In [ ]:
%%time
# hist = model.fit(X_train_image, y_train_label, epochs=epochs,
# #                  validation_split=0.1,
#                  validation_data=(X_val_image, y_val_label),
#                  verbose=1, 
#                  callbacks = [early_stopping, reduce_lr],
#                  batch_size=batch_size)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs


In [39]:

class Rand_Augment():
    def __init__(self, Numbers=None, max_Magnitude=None):
        self.transforms = ['autocontrast', 'equalize', 'rotate', 'solarize', 'color', 'posterize',
                           'contrast', 'brightness', 'sharpness', 'shearX', 'shearY', 'translateX', 'translateY']
        if Numbers is None:
            self.Numbers = len(self.transforms) // 2
        else:
            self.Numbers = Numbers
        if max_Magnitude is None:
            self.max_Magnitude = 10
        else:
            self.max_Magnitude = max_Magnitude
        fillcolor = 128
        self.ranges = {
            # these  Magnitude   range , you  must test  it  yourself , see  what  will happen  after these  operation ,
            # it is no  need to obey  the value  in  autoaugment.py
            "shearX": np.linspace(0, 0.3, 10),
            "shearY": np.linspace(0, 0.3, 10),
            "translateX": np.linspace(0, 0.2, 10),
            "translateY": np.linspace(0, 0.2, 10),
            "rotate": np.linspace(0, 360, 10),
            "color": np.linspace(0.0, 0.9, 10),
            "posterize": np.round(np.linspace(8, 4, 10), 0).astype(np.int),
            "solarize": np.linspace(256, 231, 10),
            "contrast": np.linspace(0.0, 0.5, 10),
            "sharpness": np.linspace(0.0, 0.9, 10),
            "brightness": np.linspace(0.0, 0.3, 10),
            "autocontrast": [0] * 10,
            "equalize": [0] * 10,           
            "invert": [0] * 10
        }
        self.func = {
            "shearX": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, magnitude * random.choice([-1, 1]), 0, 0, 1, 0),
                Image.BICUBIC, fill=fillcolor),
            "shearY": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, 0, magnitude * random.choice([-1, 1]), 1, 0),
                Image.BICUBIC, fill=fillcolor),
            "translateX": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, magnitude * img.size[0] * random.choice([-1, 1]), 0, 1, 0),
                fill=fillcolor),
            "translateY": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, 0, 0, 1, magnitude * img.size[1] * random.choice([-1, 1])),
                fill=fillcolor),
            "rotate": lambda img, magnitude: self.rotate_with_fill(img, magnitude),
            # "rotate": lambda img, magnitude: img.rotate(magnitude * random.choice([-1, 1])),
            "color": lambda img, magnitude: ImageEnhance.Color(img).enhance(1 + magnitude * random.choice([-1, 1])),
            "posterize": lambda img, magnitude: ImageOps.posterize(img, magnitude),
            "solarize": lambda img, magnitude: ImageOps.solarize(img, magnitude),
            "contrast": lambda img, magnitude: ImageEnhance.Contrast(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "sharpness": lambda img, magnitude: ImageEnhance.Sharpness(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "brightness": lambda img, magnitude: ImageEnhance.Brightness(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "autocontrast": lambda img, magnitude: ImageOps.autocontrast(img),
            "equalize": lambda img, magnitude: img,
            "invert": lambda img, magnitude: ImageOps.invert(img)
        }

    def rand_augment(self):
        """Generate a set of distortions.
             Args:
             N: Number of augmentation transformations to apply sequentially. N  is len(transforms)/2  will be best
             M: Max_Magnitude for all the transformations. should be  <= self.max_Magnitude """

        M = np.random.randint(0, self.max_Magnitude, self.Numbers)

        sampled_ops = np.random.choice(self.transforms, self.Numbers)
        return [(op, Magnitude) for (op, Magnitude) in zip(sampled_ops, M)]

    def __call__(self, image):
        operations = self.rand_augment()
        for (op_name, M) in operations:
            operation = self.func[op_name]
            mag = self.ranges[op_name][M]
            image = operation(image, mag)
        return image

    def rotate_with_fill(self, img, magnitude):
        #  I  don't know why  rotate  must change to RGBA , it is  copy  from Autoaugment - pytorch
        rot = img.convert("RGBA").rotate(magnitude)
        return Image.composite(rot, Image.new("RGBA", rot.size, (128,) * 4), rot).convert(img.mode)

    def test_single_operation(self, image, op_name, M=-1):
        '''
        :param image: image
        :param op_name: operation name in   self.transforms
        :param M: -1  stands  for the  max   Magnitude  in  there operation
        :return:
        '''
        operation = self.func[op_name]
        mag = self.ranges[op_name][M]
        image = operation(image, mag)
        return image

In [40]:
img_augment = Rand_Augment(Numbers=2, max_Magnitude=10)

def get_random_data(x_train_i, y_train_i, data_aug):
    x = array_to_img(x_train_i)

    if data_aug:

        seed_image = img_augment(x)
        seed_image = img_to_array(seed_image)

    else:
        seed_image = x_train_i

    seed_image = seed_image / 255

    return seed_image, y_train_i

def data_generator(x_train, y_train, batch_size, data_aug):
    '''data generator for fit_generator'''
    n = len(x_train)
    i = 0
    while True:
        image_data = []
        label_data = []
        for b in range(batch_size):
            if i==0:
                p = np.random.permutation(len(x_train))
                x_train = x_train[p]
                y_train = y_train[p]
            image, label = get_random_data(x_train[i], y_train[i], data_aug)
            image_data.append(image)
            label_data.append(label)
            i = (i+1) % n
        image_data = np.array(image_data)
        label_data = np.array(label_data)
        yield image_data, label_data

In [ ]:
%%time

hist = model.fit_generator(
                           data_generator(X_train_image, y_train_categorical, batch_size = batch_size, data_aug=True),
                           steps_per_epoch = X_train_image.shape[0] // batch_size,
                           epochs = epochs,
                           validation_data = data_generator(X_val_image, y_val_categorical, batch_size, data_aug = False),
                           validation_steps= X_val_image.shape[0] // batch_size,
                           callbacks = [early_stopping, reduce_lr],
                           shuffle = True,
                           verbose = 1)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
1250/1250 [==============================] - 198s 151ms/step - loss: 2.6732 - accuracy: 0.3474 - val_loss: 1.4468 - val_accuracy: 0.6174
Epoch 2/100
1250/1250 [==============================] - 186s 149ms/step - loss: 1.4556 - accuracy: 0.5946 - val_loss: 1.2704 - val_accuracy: 0.6534
Epoch 3/100
1250/1250 [==============================] - 186s 149ms/step - loss: 1.2347 - accuracy: 0.6521 - val_loss: 1.1350 - val_accuracy: 0.6935
Epoch 4/100
1250/1250 [==============================] - 186s 149ms/step - loss: 1.0990 - accuracy: 0.6857 - val_loss: 1.1063 - val_accuracy: 0.7024
Epoch 5/100
 904/1250 [====================>.........] - ETA: 48s - loss: 0.9948 - accuracy: 0.7173

### （５）Evaluating the model

In [ ]:
# ----------------------------------------------
# Some plots
# ----------------------------------------------
fig, (axL, axR) = plt.subplots(ncols=2, figsize=(10,4))

# loss
def plot_history_loss(fit):
    # Plot the loss in the history
    axL.plot(fit.history['loss'],label="loss for training")
    axL.plot(fit.history['val_loss'],label="loss for validation")
    axL.set_title('model loss')
    axL.set_xlabel('epoch')
    axL.set_ylabel('loss')
    axL.legend(loc='upper right')

# acc
def plot_history_acc(fit):
    # Plot the loss in the history
    axR.plot(fit.history['accuracy'],label="acc for training")
    axR.plot(fit.history['val_accuracy'],label="acc for validation")
    axR.set_title('model accuracy')
    axR.set_xlabel('epoch')
    axR.set_ylabel('accuracy')
    axR.legend(loc='upper left')

plot_history_loss(hist)
plot_history_acc(hist)
plt.show()
fig.savefig('Image Recognition_Efn.png')
plt.close()

In [ ]:
score_Efn = model.evaluate(X_val_image/255, y_val_categorical, verbose=1)
print("evaluate loss: {0[0]}".format(score_Efn))
print("evaluate acc: {0[1]}".format(score_Efn))

In [ ]:
%%time
predictions_val_Efn = model.predict(X_val_image/255)
from sklearn.metrics import log_loss
log_loss_Efn = log_loss(y_val_categorical,predictions_val_Efn)
print("EfficinetNetB0_dcr0.6_RA_logloss:{0}".format(log_loss_Efn))

### （６）Test Time Augmentation with Validation Data

In [ ]:
def tta(model,test_size,generator,batch_size ,epochs = 10):
    #test_time_augmentation
    #batch_sizeは，test_sizeの約数でないといけない．
    pred = np.zeros(shape = (test_size,100), dtype = float)
    step_per_epoch = test_size //batch_size
    for epoch in range(epochs):
        for step in range(step_per_epoch):
            sta = batch_size * step
            end = sta + batch_size
            tmp_x = generator.__next__()
            pred[sta:end] += model.predict(tmp_x)
    return pred / epochs

def tta_generator(test):
    return ImageDataGenerator(rotation_range = 0 , 
                              height_shift_range = 0.1,
                              width_shift_range = 0.1,
                              horizontal_flip = True,
                              vertical_flip = False).flow(test,batch_size = batch_size,shuffle = False)


In [ ]:
%%time
batch_size = 20
tta_epochs = 20
tta_pred_Efn = tta(model,X_val_image.shape[0],tta_generator(X_val_image/255),batch_size,epochs = tta_epochs)
from sklearn.metrics import log_loss
TTA_log_loss_Efn = log_loss(y_val_categorical,tta_pred_Efn)

print( accuracy_score( np.argmax(tta_pred_Efn,axis = 1) , np.argmax(y_val_categorical,axis = 1)))
print( "TTA_EfficinetNetB0_dcr0.6_RA_logloss:{0}".format(TTA_log_loss_Efn))

### （７）Predicting Image Classification

In [ ]:
%%time
batch_size = 20
tta_epochs = 50
tta_prediciton_Efn = tta(model,test.shape[0],tta_generator(test/255),batch_size ,epochs = tta_epochs)

In [ ]:
file_name = sample[0]
df = pd.DataFrame(tta_prediciton_Efn)

In [ ]:
df_20 = pd.DataFrame()
df_20[0] = df[4] + df[30] + df[55] + df[72] + df[95]
df_20[1] = df[1] + df[32] + df[67] + df[73] + df[91]
df_20[2] = df[54] + df[62] + df[70] + df[82] + df[92]
df_20[3] = df[9] + df[10] + df[16] + df[28] + df[61]
df_20[4] = df[0] + df[51] + df[53] + df[57] + df[83]
df_20[5] = df[22] + df[39] + df[40] + df[86] + df[87]
df_20[6] = df[5] + df[20] + df[25] + df[84] + df[94]
df_20[7] = df[6] + df[7] + df[14] + df[18] + df[24]
df_20[8] = df[3] + df[42] + df[43] + df[88] + df[97]
df_20[9] = df[12] + df[17] + df[37] + df[68] + df[76]
df_20[10] = df[23] + df[33] + df[49] + df[60] + df[71]
df_20[11] = df[15] + df[19] + df[21] + df[31] + df[38]
df_20[12] = df[34] + df[63] + df[64] + df[66] + df[75]
df_20[13] = df[26] + df[45] + df[77] + df[79] + df[99]
df_20[14] = df[2] + df[11] + df[35] + df[46] + df[98]
df_20[15] = df[27] + df[29] + df[44] + df[78] + df[93]
df_20[16] = df[36] + df[50] + df[65] + df[74] + df[80]
df_20[17] = df[47] + df[52] + df[56] + df[59] + df[96]
df_20[18] = df[8] + df[13] + df[48] + df[58] + df[90]
df_20[19] = df[41] + df[69] + df[81] + df[85] + df[89]
df_concat = pd.concat([file_name, df_20],axis=1)
df_concat.head()

In [ ]:
# file_name = sample[0]
# df = pd.DataFrame(tta_prediciton_Efn)
# df_concat = pd.concat([file_name, df],axis=1)
# df_concat.head()

In [81]:
df_concat.to_csv('/content/drive/MyDrive/AI Quest/Submit/submit_TTA_EffNetB0_dcr0.2_RA_Logloss{0}.csv'.format(TTA_log_loss_Efn),index = False, header=None)